In [ ]:
from pathlib import Path

import re

import pandas as pd
import numpy as np

import biopsykit as bp
import biopsykit.saliva as saliva

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
path = Path("../example_data")

## Load Condition List

In [ ]:
condition_list = bp.io.load_subject_condition_list(path.joinpath("condition_list.csv"), return_dict=False)
condition_list.head()

## Load Data

### Option 0: Load BioPsyKit example data 

In [ ]:
df_cort = bp.example_data.get_saliva_example()

#### Example: Exclude Subjects 'Vp01' and 'Vp02' from Condition List and Cortisol DataFrame

In [ ]:
dict_result = bp.utils.exclude_subjects(['Vp01', 'Vp02'], condition_list=condition_list, cortisol=df_cort)

dict_result
# reassign cleaned data
# df_cort = dict_result['cortisol']
# condition_list = dict_result['condition_list']

### Option 1: Use BioPsyKit to load saliva data in 'plate' format

Load Data into pandas Dataframe

In [ ]:
cort_path = path.joinpath("cortisol_sample_plate.xlsx")
df_cort = saliva.io.load_saliva_plate(file_path=cort_path, biomarker_type="cortisol", sample_id_col="sample ID", data_col="cortisol (nmol/l)", condition_list=condition_list)

In [ ]:
df_cort

Save Dataframe as csv (in standardized format)

In [ ]:
saliva.io.save_saliva(path.joinpath("cortisol_sample.csv"), df_cort)

### Option 2: Use BioPsyKit to load saliva data that's already in the 'correct' format

In [ ]:
cort_path = path.joinpath("cortisol_sample.csv")
df_cort = saliva.io.load_saliva(file_path=cort_path, biomarker_type="cortisol")
df_cort.head()

## Plot Data (some very simple Examples with Seaborn)

In [ ]:
fig, ax = plt.subplots(figsize=(7,5))
sns.set_theme(style='ticks')
sns.lineplot(data=df_cort.reset_index(), x='sample', y='cortisol', hue='condition', ci=None, ax=ax);
ax.set_xticks(df_cort.index.get_level_values('sample').unique())
ax.set_xticklabels(["S{}".format(i) for i in df_cort.index.get_level_values('sample').unique()])
ax.legend().remove()
ax.set_ylabel("Cortisol [nmol/l]")
ax.set_xlabel("Messzeitpunkte")
fig.tight_layout()
fig.savefig("/Users/Richer/Desktop/cortisol_example.pdf", transparent=True)

In [ ]:
sns.relplot(data=df_cort.reset_index(), x='sample', y='cortisol', kind='line', hue='condition')

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(data=df_cort.reset_index(), x='sample', y='cortisol', hue='condition', ax=ax)

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(data=saliva.max_increase(df_cort).reset_index(), x='condition', y='cortisol_max_inc')

In [ ]:
fig, ax = plt.subplots()

display(saliva.standard_features(df_cort).groupby('condition').mean())

data_long = pd.wide_to_long(saliva.standard_features(df_cort).reset_index(), stubnames="cortisol", sep='_', i=['subject', 'condition'], j='feature', suffix=r"\w+")
sns.boxplot(data=data_long.reset_index(), x='feature', y='cortisol', hue='condition', ax=ax);